In [1]:
!pip install sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 8.6 MB/s eta 0:00:

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
import os


In [3]:
from google.colab import files
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
print(f"Uploaded file: {pdf_path}")


Saving Day-10  Vector DB.pdf to Day-10  Vector DB.pdf
Uploaded file: Day-10  Vector DB.pdf


In [4]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print("Total pages loaded:", len(documents))


Total pages loaded: 11


In [5]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_documents(documents)
print("Total chunks created:", len(chunks))


Total chunks created: 22


In [6]:
print(chunks[0].page_content[:400])


preencoded.png
Day 10
Vector Databases


In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipython-input-2734244670.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
faiss_db = FAISS.from_documents(chunks, embedding_model)
print("FAISS Vector DB created!")


FAISS Vector DB created!


In [10]:
query = "What is vector data base"
results = faiss_db.similarity_search(query, k=3)

for r in results:
    print(r.page_content[:200])
    print("---")


preencoded.png
Why Vector Databases are Essential
At the core of advanced information retrieval, vector databases revolutionize how we interact 
with data by enabling semantic search. This process tra
---
preencoded.png
Key Takeaways: The Power of 
Vector Databases
Vector databases are transformative technologies, enabling AI systems to 
understand and retrieve information based on meaning, rather than
---
preencoded.png
Day 10
Vector Databases
---


In [11]:
chroma_dir = "./chroma_db"
if not os.path.exists(chroma_dir):
    os.makedirs(chroma_dir)

chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory=chroma_dir
)

print("Chroma DB created and persisted!")


Chroma DB created and persisted!


In [13]:
query = "what is meant by FAISS and ChormaDB?"
results = chroma_db.similarity_search(query, k=3)

for r in results:
    print(r.page_content[:200])
    print("---")


Pinecone Cloud managed Large-scale production, fully managed service
Weaviate Hybrid search Combines graph & vector search for complex queries
Today's focus: FAISS and Chroma for their local and devel
---
preencoded.png
Leading Vector Store Solutions
The landscape of vector databases offers diverse options, each suited for different scales and use cases. We'll focus 
on FAISS and Chroma, key players in
---
preencoded.png
FAISS: Facebook AI Similarity 
Search
Developed by Facebook AI Research, FAISS is an open-source library that 
provides highly optimized algorithms for efficient similarity search and 

---
